In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [2]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")

uv_train, uv_test = train_test_split(user_vectors, test_size=0.2, random_state=8)

ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [3]:
encoder = LabelEncoder()
t_encoder = LabelEncoder()
final_vectors = uv_train.copy()
test_vectors = uv_test.copy()
x = encoder.fit(final_vectors["username"])
t = t_encoder.fit(test_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
test_enc = t.transform(test_vectors["username"])

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc
test_vectors["username"] = test_enc
#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
test_arrays = test_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#print(encoder.inverse_transform([1675]))
final_vectors
#uv_train
#test_arrays
#test_vectors
#uv_test

,username,experience,gender,generation,fav_anime_period,Shounen,Seinen,Mystery,Kids,Parody,...,Harem,Psychological,Samurai,Magic,Thriller,Slice of Life,Mecha,Vampire,Shounen Ai,Comedy
7053,6490,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4363,4346,3,1,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8451,3028,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1478,2946,2,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8047,1550,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5480,3154,1,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6995,1675,2,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2181,4684,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2033,245,3,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Compute K-Nearest Neighbours

In [125]:
#function that returns a list of indexes of most similar users in "final_arrays" array
def neighbours(user):
    #user = np.array([7537, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    nc = NearestNeighbors(n_neighbors = 5, metric="euclidean") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data (training)
    n = train.kneighbors([user], return_distance = False)
    return n

#returns a dataframe of users that were in k-nearest neighbours list 
def sim_frame(neighbours):
    sim_names = []
    #neighbours = neighbours()

    for i in neighbours[0]: #loop through nested array
        #print(final_arrays[i][0])
        name = x.inverse_transform([final_arrays[i][0]]) #find usernames of similar users
        sim_names.append(name)

    #build new dataframe of neighbours
    neighbour_frame = pd.DataFrame()
    for i in users.columns:
        neighbour_frame[i] = None

    for name in sim_names:
        user_row = users.loc[users["username"]==name[0]]
        neighbour_frame = pd.concat([neighbour_frame, user_row])

    pd.options.display.max_rows = 999
    pd.set_option('display.max_columns', 500)
    return neighbour_frame
p = [1675,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

neighbour_ids = neighbours(np.array(p))
#print(neighbour_ids)
neighbour_frame = sim_frame(neighbour_ids)
neighbour_frame
#final_vectors.loc[final_vectors["username"]==7537]
#(final_vectors.columns)    

,user_id,username,user_completed,stats_mean_score,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
6995,233922,Harmleikur,170,8.09,Regular,Female,1992,4256,Millenials,"['Drama', 'Comedy', 'Adventure']",Classic
3756,380488,Harmony_,41,7.71,Newbie,Female,1985,1619,Millenials,"['Adventure', 'Action', 'Drama']",Classic
8501,4568002,Harish123uzumaki,134,7.76,Regular,Male,1997,4473,Gen Z,"['Shounen', 'Adventure', 'Drama']",Classic
9092,65855,Harima_Kenji,92,7.26,Regular,Male,1981,3204,Millenials,"['Adventure', 'Comedy', 'Sci-Fi']",Classic
2244,3547033,Haroun_SMIDA,343,7.07,Veteran,Male,1991,7978,Millenials,"['Shounen', 'Comedy', 'Mystery']",New-Gen


Create anime recommendations

In [126]:

#Put anime that have most members at the top of each users "lists"
def recommendations(neighbours, media_type):
    #Get top rated TV shows or movies from each user
    suggestions = [] #anime_id's of recommended anime
    for i in neighbours["username"]:
        user_ratings = ratings.loc[ratings["username"]==i]
        tv_shows = user_ratings[user_ratings["type"]=="TV"]
        movies = user_ratings[user_ratings["type"]=="Movie"]

        if media_type == "TV":
            show_ratings = tv_shows
        else:
            show_ratings = movies
        
        #There may be 0 movies in the user's top 10 list in which case we skip that user's recommendationan
        if len(show_ratings) > 1:
            top_rated = show_ratings.iloc[0]["anime_id"] #change to ids when creating the website to lookup the anime
            suggestions.append(top_rated)
            #print(top_rated)

    if len(suggestions)==0:
        print("No movies to recommend :/")
    
    return suggestions

recommendations(neighbour_frame, "TV")
#ratings

[30276, 486, 1735, 237, 1535]

Based on model fit with training data (K-Neighbours), evaluate recommended anime for users in test set


In [6]:
real_ratings = pd.read_csv(r"D:\dataset\animelists_cleaned.csv")

In [173]:
user_fav_genres = set(eval(users.loc[users["username"]=="karthiga"]["fav_genres"][0]))
user_fav_genres

a_genres = list(anime.loc[anime["title"]=="Junjou Romantica 2"]["genre"])
a_genres = a_genres[0].split(",")
a_genres = set([x.strip() for x in a_genres])
len(user_fav_genres.intersection(a_genres))

2

In [7]:
def evaluate_users():
    #Create kneighbours dataframe for all users in the dataset
    overall_scores = []
    num_likes = []
    num_shows_watched = []
    for user in test_arrays:
        user = np.array(user)
        #print([user[0]])
        username = t.inverse_transform([user[0]])[0]
        #print(username)
        user_fav_genres = set(eval(users.loc[users["username"]==username]["fav_genres"][0]))
        user_neighbours = neighbours(user)
        nframe = sim_frame(user_neighbours)
        #print(user_neighbours)
        user_recommendations = recommendations(nframe, "TV")
        #print(nframe["username"], user_recommendations)
        
        #Use original ratings dataframe with all shows user has rated instead of simply the top 10
        rt = real_ratings.loc[real_ratings["username"]==username]
        rt = rt.loc[rt["my_watched_episodes"]>3]
        score = 0
        likes = 0
        shows_watched = 0
        #Check whether recommended shows are present in original ratings table
        for show in user_recommendations:
            a_id = show
            #a_id = int(anime.loc[anime["title"]==show]["anime_id"])

            #Find how many genres in recommended anime are in the user's favourites
            a_genres = list(anime.loc[anime["anime_id"]==a_id]["genre"])
            a_genres = a_genres[0].split(",")
            a_genres = set([x.strip() for x in a_genres])

            num_matched_genres = len(user_fav_genres.intersection(a_genres)) #max = 3
            #score+=num_matched_genres
            if int(a_id) in list(rt["anime_id"]): #if user watched the recommended show, give 1 point
                rating_row = real_ratings.loc[(real_ratings["username"]==username)&(real_ratings["anime_id"]==a_id)]
                #print(rating_row)
                shows_watched+=1
                score+=1 #add a point to overall score because user has watched recommended show before
                if int(rating_row["my_score"])>=7:#user "likes" the show if they rated it higher than or equal to 7
                    score+=1 #add an extra point to overall score if the user liked the show
                    likes+=1

        #max number of points is 2 x number of neighbours (k): 10 + max amount fo matched genres (3)
        #max_score = 13
        num_likes.append(likes)
        overall_scores.append(score)
        num_shows_watched.append(shows_watched)
        #print(f"Score for user {username}: {score}%")

        #standard deviation of results
        #distribution of scores - how many score are 1,2,3,4,5,6,7,8,9,10
        #plot on graph
        #potentially take top 2 instead of top 1


    avg_score = np.around(np.average(overall_scores),decimals=2)
    avg_shows_watched = np.around(np.average(num_shows_watched),decimals=2)
    avg_likes = np.around(np.average(num_likes),decimals=2)
    
    print(f"Average overall recommendation score for Test users: {avg_score} | Standard deviation: {np.std(overall_scores)}")
    print(f"Average number of recommended shows that users have watched before: {avg_shows_watched} | Standard deviation: {np.std(num_shows_watched)}")
    print(f"Average number of recommended anime that users liked: {avg_likes} | Standard deviation: {np.std(num_likes)}")
    return overall_scores, num_likes, num_shows_watched, avg_score, avg_shows_watched, avg_likes

results = evaluate_users() #147minutes
#include analysis of distance metric between neighbours e.g. cosine, euclidean 

Average overall recommendation score for Test users: 3.9666845781837723 | Standard deviation: 2.4823865772017375
Average number of recommended shows that users have watched before: 2.1719505642127888 | Standard deviation: 1.2929303531695675
Average number of recommended anime that users liked: 1.7947340139709833 | Standard deviation: 1.290561860515719


(3.9666845781837723, 2.1719505642127888, 1.7947340139709833)

In [ ]:
from matplotlib import pyplot as plt

scores = results[0]
x = list(range(1, len(scores)+1))

Collaborative filtering for additional suggestions

In [17]:
rating_vectors = pd.read_csv(r"D:\dataset\encoding\collab_scores.csv")
anime_frame = pd.read_csv(r"D:\dataset\encoding\anime_frame.csv")
rv_train, rv_test = train_test_split(rating_vectors, test_size=0.2, random_state=1)

In [91]:
def r_neighbours(anime):
    nc = NearestNeighbors(n_neighbors = 2, metric="cosine") #euclidean #find 2 most similar anime to each of the inital 5 recommendations
    train = nc.fit(rv_train.values) #fit the model with the ratings data
    n = train.kneighbors([anime], return_distance = False)
    return n
print(len(rv_test))

630


In [96]:

def r_neighbour_ids(initial):
    n_ids = []
    print(initial)
    for r in initial:
        #a_id = int(anime.loc[anime["title"]==r]["anime_id"])
        a_vector = list(rating_vectors.loc[rating_vectors["anime_id"]==r].values)
        n_indexes = r_neighbours(a_vector[0])[0]
        #print(n_indexes)
        for i in n_indexes:
            anime_id = rating_vectors.iloc[i]["anime_id"]
            n_ids.append(anime_id)

    print(n_ids)

    return n_ids
#r_neighbour_ids([30276, 11665, 10162, 1293, 245])   
#user_ratings[user_ratings["type"]=="TV"]
def get_additional_anime(initial):
    more_recs = pd.DataFrame()
    for i in r_neighbour_ids(initial):
        #print(i)
        anime_row = anime_frame.loc[anime_frame["anime_id"]==i]
        #print(anime_row)
        
        more_recs = pd.concat([more_recs, anime_row])
        more_recs = more_recs.loc[:,["anime_id","title","studio", "genre","time_period", "fame"]]
    #exclude the first anime (the show being compared to)
    #more_recs=more_recs[1:]
    return more_recs
#get_additional_anime()


[30276, 11665, 10162, 1293, 245]
[25731, 34126, 34428, 2547, 32011, 8364, 6347, 3750, 14653, 54]


In [ ]:
def evaluate_ratings():
    for a in rv_test.values:
        anime_ids = r_neighbour_ids(a)
        print(anime_ids, len(anime_ids))

        k_user_ids = neighbours(a)
        k_neighbour_frame = sim_frame(k_user_ids)
        main_recs = recommendations(k_neighbour_frame,"TV")
        #print(main_recs)
        #Additional anime recommendations
        more_anime = get_additional_anime(main_recs)

#evaluate raw scores and normalised
        


#evaluate_ratings()

In [95]:
def full_recommendations():
    #Initial anime recommendations
    sample_user = [1675,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
    k_user_ids = neighbours(sample_user)
    k_neighbour_frame = sim_frame(k_user_ids)
    main_recs = recommendations(k_neighbour_frame,"TV")
    #print(main_recs)
    #Additional anime recommendations
    more_anime = get_additional_anime(main_recs)
    
    return main_recs, more_anime
full_recommendations()[1]

[30276, 11665, 10162, 1293, 245]
[25731, 34126, 34428, 2547, 32011, 8364, 6347, 3750, 14653, 54]


,anime_id,title,studio,genre,time_period,fame
408,34126,Rewrite 2nd Season,8bit,"['Action', ' Comedy', ' Supernatural', ' Roman...",New-Gen,well-known
2093,34428,Overlord Movie 2: Shikkoku no Eiyuu,Madhouse,"['Action', ' Game', ' Supernatural', ' Magic',...",New-Gen,niche
1496,2547,Princess Sara,Nippon Animation,"['Slice of Life', ' Historical', ' Drama', ' S...",Classic,niche
1638,32011,Hakuouki: Otogisoushi,DLE,"['Fantasy', ' Historical', ' Josei', ' Samurai']",New-Gen,unknown
552,8364,Crayon Shin-chan Movie 12: Arashi wo Yobu! Yuu...,Shin-Ei Animation,"['Action', ' Adventure', ' Comedy', ' Historic...",Classic,unknown
353,6347,Baka to Test to Shoukanjuu,Silver Link.,"['Comedy', ' Romance', ' School', ' Super Power']",New-Gen,famous
1072,3750,Maria-sama ga Miteru 4th,Studio Deen,"['School', ' Shoujo', ' Shoujo Ai', ' Slice of...",Classic,niche
341,14653,Hayate no Gotoku! Can&#039;t Take My Eyes Off You,Manglobe,"['Comedy', ' Harem', ' Parody', ' Shounen']",New-Gen,well-known
540,54,Appleseed (Movie),Digital Frontier,"['Action', ' Military', ' Sci-Fi', ' Police', ...",Classic,well-known


Function that converts user information from website into vector